In [1]:
%pwd

'c:\\Users\\Dzuels Foundation\\mlproject\\notebook'

In [2]:
%cd ..\

c:\Users\Dzuels Foundation\mlproject


In [11]:
import requests
import os
import sys
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from joblib import dump, load
import mlflow
import dagshub
import mlflow.sklearn
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score


from src.constant import *
from src.logger import logging
from src.exception import CustomException
from src.utils import read_yaml,create_directory,save_json

In [5]:

dagshub.init(repo_owner='harcuracy', repo_name='machine_learning_project', mlflow=True)

[2025-04-08 16:19:38,684] : httpx : INFO: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as harcuracy

[2025-04-08 16:19:38,701] : dagshub : INFO: Accessing as harcuracy
[2025-04-08 16:19:39,487] : httpx : INFO: HTTP Request: GET https://dagshub.com/api/v1/repos/harcuracy/machine_learning_project "HTTP/1.1 200 OK"
[2025-04-08 16:19:40,314] : httpx : INFO: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "harcuracy/machine_learning_project"

[2025-04-08 16:19:40,327] : dagshub : INFO: Initialized MLflow to track repo "harcuracy/machine_learning_project"


Repository harcuracy/machine_learning_project initialized!

[2025-04-08 16:19:40,331] : dagshub : INFO: Repository harcuracy/machine_learning_project initialized!


In [ ]:
@dataclass (frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    model_file:Path
    test_data_file:Path
    report_file:Path
    target_column: str
    mlflow_tracking_uri: str
    all_params: dict
    


In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)
                
        create_directory(self.config.artifact_root)
        
    def get_evaluation_config(self)-> ModelEvaluationConfig:
        config = self.config.model_evaluation
        target_column = self.schema.TARGET_COLUMN.name
        all_param = self.params.PARAMETERS.model.params
        
        create_directory(config.root_dir)
        
        model_trainer_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_file=config.model_file,
            test_data_file=config.test_data_file,
            report_file=config.report_file,
            target_column=target_column,
            all_params=all_param,
            mlflow_tracking_uri=config.mlflow_tracking_uri
        )
        return model_trainer_config

In [ ]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metric(self,y_true,y_pred):
        rmse = root_mean_squared_error(y_true, y_pred, squared=False)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        model = load(self.config.model_file)
        test_data = pd.read_csv(self.config.test_data_file)
        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]
        
        mlflow.set_tracking_uri(self.config.mlflow_tracking_uri)
        mlflow.set_experiment(experiment_name="model_evaluation")
        
        with mlflow.start_run():
            y_pred = model.predict(X_test)
            rmse,mae, r2 = self.eval_metric(y_test, y_pred)
            scores = {
                "rmse" : rmse,
                "mae": mae,
                "r2": r2
            }
            save_json( scores, self.config.report_file)
            mlflow.log_params(self.config.all_params)
    
            mlflow.log_metrics(scores)
            mlflow.sklearn.log_model(model, artifact_path="model_training", registered_model_name="ElasticNet")
            

    


In [ ]:
try:
    config = ConfigurationManager()
    model_evalution_config = config.get_evaluation_config()
    model_evalution = ModelEvaluation(config=model_evalution_config)
    model_evalution.log_into_mlflow()
    
except Exception as e:
    logging.error(f"Error in model evaluation: {str(e)}")
    raise CustomException(e, sys) from e

[2025-04-08 16:20:09,442] : root : INFO: yaml file: config\config.yaml loaded successfully
[2025-04-08 16:20:09,448] : root : INFO: yaml file: schema.yaml loaded successfully
[2025-04-08 16:20:09,453] : root : INFO: yaml file: params.yaml loaded successfully
[2025-04-08 16:20:09,454] : root : INFO: Directory 'artifact' already exists.
[2025-04-08 16:20:09,456] : root : INFO: Directory 'artifact/model_evaluation' already exists.
[2025-04-08 16:20:11,264] : root : INFO: json file: artifact/model_evaluation/report.json saved successfully


2025/04/08 16:20:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNet' already exists. Creating a new version of this model...
2025/04/08 16:20:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNet, version 2
Created version '2' of model 'ElasticNet'.


🏃 View run enthused-chimp-124 at: https://dagshub.com/harcuracy/machine_learning_project.mlflow/#/experiments/0/runs/cf8021ead77b4a1eb1c03ca1a0fd0460
🧪 View experiment at: https://dagshub.com/harcuracy/machine_learning_project.mlflow/#/experiments/0
